---
title: Calcium imaging in the zebrafish tectum
description: Are direction-selective cells in the zebrafish optical tectum colorblind?
author: 'Alexander Wendt, Patrick Weygoldt'
date: '2018-05-04'
date-format: long
published-title: Date
author-title: Authors
title-block-banner: '#1d1d1d'
format:
  html:
    code-fold: false
    fig-format: svg
    fig-responsive: true
    theme:
      light: flatly
      dark: darkly
jupyter: python3
---

In [ ]:
#| output: false

import os
import matplotlib.pyplot as plt
import numpy as np
import modules.functions as fs
from dataclasses import dataclass
from vxtools.summarize.structure import SummaryFile
from sklearn.metrics import auc
from scipy.stats import pearsonr
from modules.dataloader import all_rois
from modules.plotstyle import PlotStyle

ps = PlotStyle()